# Работа с данными формата HDF5

HDF5 (Hierarchical Data Format, HDF (Иерархический формат данных)) - бинарный формат, который позволяет эффективно хранить большие объемы данных, реализация в Python имеет хорошую интеграцию с библиотекой векторных вычислений numpy. 

Основые преимущества формата:

* хранение сложных иерархических структур
* быстрый доступ к изолированным частям данных без загрузки в память (чтение с диска)

Реализация в python называется [h5py](https://www.h5py.org/). Это довольно низкоуровневая библиотека, поэтому рекомендую дополнительно изучить удобную в использовании [PyTables](https://www.pytables.org/), которая является оберткой над h5py. Кроме того, данные в hdf позволяет сохранять библиотека для работы с табличными данными [pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_hdf.html)

HDF спроектирован для эффективного хранения многомерных массивов. 

Для примера работы получим три массива numpy и сохраним их в формат HDF.

In [1]:
from scipy.sparse import random
import numpy as np
from scipy.sparse.linalg import svds

# формируем массив случайных чисел
user_item_matrix = random(10000, 1000, density=0.01, format='coo', dtype=np.int32, random_state=42)
user_item_matrix = user_item_matrix.asfptype()
# разделяем массив на три части, каждую из которых будем хранить отдельно
U, s, V = svds(user_item_matrix, k=40)

print("Размерности матриц U={}, s={}, V={}".format(U.shape, s.shape, V.shape))
print("Типы матриц U={}, s={}, V={}".format(type(U), type(s), type(V)))

Размерности матриц U=(10000, 40), s=(40,), V=(40, 1000)
Типы матриц U=<class 'numpy.ndarray'>, s=<class 'numpy.ndarray'>, V=<class 'numpy.ndarray'>


Логической единицей хранения в hdf является dataset. Работа с датасетом не отличается от работы с текстовыми файлами, которые мы изучили ранее в курсе: мы создаём менеджер контекста **h5py.File** и загружаем датасет внутри контеста. За пределами менеджера контекста датасет доступен не будет

In [2]:
import h5py

with h5py.File('s_matrix.hdf5', 'w') as f:
    h5py_dset = f.create_dataset("default", data=s)
    print('Создали датасет в менеджере контекста: {}'.format(h5py_dset))

print('Датасет вне менеджера контекста недоступен: {}'.format(h5py_dset))

OSError: Unable to create file (unable to open file: name = './data/s_matrix.hdf5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

Киллерфича hdf5 - сохранение датасет сложной структуры. Ближаший аналог - модуль pickle умеет сохранять только питоновские объекты, в то время как h5py способен хранить "разветвлённые" иерархические данные.

Сохраним три отдельных numpy.array в едином датасете:

In [6]:
with h5py.File('complex_dataset.hdf5', 'w') as f:
    # создаём группу raw
    raw = f.create_group('source_data')
    raw.create_dataset('complex_dataset.hdf5', data=np.random.random(1000))
    # создаём подгруппу processed
    processed = raw.create_group('model_data')
    # в подгруппе proceessed группы raw создаём датасет
    processed.create_dataset('user_factors', data=U, dtype=np.float32, compression="gzip")
    processed.create_dataset('eigen_values', data=s, dtype=np.float32, compression="gzip")
    processed.create_dataset('item_factors', data=V, dtype=np.float32, compression="gzip")
    
    print("Уровень группы:\t\t{}".format(raw.items()))
    print("Уровень подгруппы\t{}".format(processed.items()))
    # формируем список элементов, которые есть в подгруппе
    print("\nИмена элементов в подгруппе:\n")
    group_names = [i.name for i in f['source_data/model_data'].values()]
    for name in group_names:
        print(name)

Уровень группы:		ItemsViewHDF5(<HDF5 group "/source_data" (2 members)>)
Уровень подгруппы	ItemsViewHDF5(<HDF5 group "/source_data/model_data" (3 members)>)

Имена элементов в подгруппе:

/source_data/model_data/eigen_values
/source_data/model_data/item_factors
/source_data/model_data/user_factors


In [0]:
!ls -hla ./data | grep hdf5

Ниже мы сохраням все результаты разложения в директорию `/data/`. Когда мы открываем датасет на чтение, он доступен для применения различных функций (вроде min, max), но при этом он теряет свойства numpy.ndarray. Чтобы перейти обратно к массиву можно сделать slice - мы итеративно пройдёмся по данным на диске и "соберём" их обратно в numpy.ndarray.

In [7]:
with h5py.File('s_matrix.hdf5', 'r') as f:
    print("Доступные ключи %s\n" % list(f.keys()))
    data = f['default']
    print("min={}, \nmax={}, \nslice={}\n".format(min(data), max(data), data[10:15]))
    print("Попытка воспользоваться функциями numpy...")
    try:
        print(data.min())
    except AttributeError:
        print("Не вышло =(")
    data_copy = data[:]
print("Типизация датасетов: data={}, data_copy={}\n".format(data, type(data_copy)))

# Чтение иерархических датасетов
with h5py.File('complex_dataset.hdf5', 'r') as f:
    U_hdf = f['source_data/model_data/user_factors'][:]
    s_hdf = f['source_data/model_data/eigen_values'][:]
    V_hdf = f['source_data/model_data/item_factors'][:]
    
print("Размерности матриц U={}, s={}, V={}".format(U_hdf.shape, s_hdf.shape, V_hdf.shape))
print("Типизация матриц U={}, s={}, V={}".format(type(U_hdf), type(s_hdf), type(V_hdf)))

Доступные ключи ['default']

min=15967970843.644163, 
max=16869307715.289265, 
slice=[1.61010982e+10 1.61161420e+10 1.61308403e+10 1.61512488e+10
 1.61722312e+10]

Попытка воспользоваться функциями numpy...
Не вышло =(
Типизация датасетов: data=<Closed HDF5 dataset>, data_copy=<class 'numpy.ndarray'>

Размерности матриц U=(10000, 40), s=(40,), V=(40, 1000)
Типизация матриц U=<class 'numpy.ndarray'>, s=<class 'numpy.ndarray'>, V=<class 'numpy.ndarray'>
